<a href="https://colab.research.google.com/github/SanjaySaatyaki/hf_smol_course/blob/main/Preference_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>